In [15]:
# imports
import sys
sys.path.append('../')

from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from PIL import Image
from src.load_data import master_load
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Activation, BatchNormalization, Conv1D, Conv2D,
                                     Dense, Dropout, Flatten, LSTM, MaxPooling1D,
                                     MaxPooling2D, Reshape, TimeDistributed)
from tensorflow.keras import regularizers
from tensorflow.math import confusion_matrix
import time


from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K

In [2]:
# set plotting params
plt.rcParams['font.serif'] = 'Ubuntu'
plt.rcParams['font.monospace'] = 'Ubuntu Mono'
plt.rcParams['font.size'] = 32
plt.rcParams['axes.labelsize'] = 32
plt.rcParams['axes.titlesize'] = 32
plt.rcParams['xtick.labelsize'] = 24
plt.rcParams['ytick.labelsize'] = 24
plt.rcParams['legend.fontsize'] = 32
plt.rcParams['figure.titlesize'] = 48

In [4]:
model_df = pd.DataFrame({'loss': [], 'accuracy': [], 'val_loss': [], 'val_accuracy': [], 'model_name': []})

In [5]:
def fit_model(model, X_train, y_train, X_test, y_test, epochs=5, batch_size=32, model_type='numpy'):
    global model_df
    
    fitted = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))
    score = fitted.model.evaluate(X_test, y_test, batch_size=batch_size)

    model_name = f'../models/{model_type}/acc_{round(score[1], 3)}-loss_{round(score[0], 2)}-{int(time.time())}.model'

    temp_df = pd.DataFrame.from_dict(fitted.history)
    temp_df['model_name'] = model_name
    model_df = pd.concat([model_df, temp_df])

    fitted.model.save(model_name)
    print('done')

In [12]:
# load fft data
X_train, X_test, y_train, y_test = master_load()

In [68]:
model = Sequential()

model.add(Conv1D(512, (5), padding='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))

model.add(Conv1D(256, (5), padding='same'))
model.add(Activation('relu'))

model.add(Conv1D(128, (5), padding='same'))
model.add(Activation('relu'))

model.add(Conv1D(64, (5), padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Conv1D(3, (16)))
model.add(Reshape((3,)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_57 (Conv1D)           (None, 16, 512)           154112    
_________________________________________________________________
activation_65 (Activation)   (None, 16, 512)           0         
_________________________________________________________________
conv1d_58 (Conv1D)           (None, 16, 256)           655616    
_________________________________________________________________
activation_66 (Activation)   (None, 16, 256)           0         
_________________________________________________________________
conv1d_59 (Conv1D)           (None, 16, 128)           163968    
_________________________________________________________________
activation_67 (Activation)   (None, 16, 128)           0         
_________________________________________________________________
conv1d_60 (Conv1D)           (None, 16, 64)          

In [ ]:
fit_model(model, X_train, y_train, X_test, y_test, epochs=100, batch_size=32, model_type='numpy')

Epoch 1/100
8895/8895 [==============================] - 198s 22ms/step - loss: 0.7452 - accuracy: 0.6332 - val_loss: 1.7972 - val_accuracy: 0.4569
Epoch 2/100
8895/8895 [==============================] - 198s 22ms/step - loss: 0.2700 - accuracy: 0.8950 - val_loss: 3.0420 - val_accuracy: 0.4275
Epoch 3/100
8895/8895 [==============================] - 196s 22ms/step - loss: 0.1665 - accuracy: 0.9397 - val_loss: 3.5634 - val_accuracy: 0.4263
Epoch 4/100
8895/8895 [==============================] - 194s 22ms/step - loss: 0.1455 - accuracy: 0.9499 - val_loss: 3.8883 - val_accuracy: 0.4165
Epoch 5/100
8895/8895 [==============================] - 194s 22ms/step - loss: 0.1158 - accuracy: 0.9608 - val_loss: 5.9415 - val_accuracy: 0.4209
Epoch 6/100
8895/8895 [==============================] - 193s 22ms/step - loss: 0.1150 - accuracy: 0.9627 - val_loss: 5.8408 - val_accuracy: 0.4180
Epoch 7/100
8895/8895 [==============================] - 186s 21ms/step - loss: 0.1062 - accuracy: 0.9665 - val_

In [ ]:
# 8895/8895 [==============================] - 245s 28ms/step - loss: 0.9311 - accuracy: 0.5686 - val_loss: 1.1161 - val_accuracy: 0.4921
model = Sequential()

model.add(Conv1D(64, (5), padding='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))

model.add(Conv1D(128, (5), padding='same'))
model.add(Activation('relu'))

model.add(Conv1D(256, (5), padding='same'))
model.add(Activation('relu'))

model.add(Conv1D(512, (5), padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Conv1D(3, (16)))
model.add(Reshape((3,)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()